Import the NIH Toolbox Data Dictionary into a Pandas Dataframe and interpret some of the responses

In [1]:
 
#Here is the link: https://nihtoolbox.desk.com/customer/portal/kb_article_attachments/144125/original.xlsx?1562788410

# This notebook provides a solution to the Button problem and creates the 'interpretted' NIH Toolbox data dictionary on this repository

The Button problem:
In short the button problem is a lack of transparency between the published NIH Toolbox Data dictionary and the actual output from the IPADS for variables that encode 'correct' or 'incorrect' answers from a selection of possibilities.  

Open the NIH Toolbox Data dictionary, either from the link above, or from the copy stored on this repo

look at an example, say for 'NIH Toolbox Dimensional Change Card Sort Test Age 12+ v2.1'
You will notice that the bulk of items for this Instrument have something like the following examples in the Response column

example 1:
0=BlueBall\n1=YellowTruck\n  

example 2:
1=BlueBall\n0=YellowTruck\n 

This is confusing, because the ACTUAL data exported contains 1's and 2's.  Meanwhile, the NDA fields are prepared to accept 'correct' and 'incorrect'


We have been instructed (NIH Toolbox help desk) to interpret this notation as follows: 
A '1' on the left of the '=' always means that selecting the item to the right of the '=' is 'correct', 
0 on the left of the '='  means that selecting the item to the right of the '='  is 'incorrect'

The first item listed in the Responses (Blue Ball in this case) corresponds with a '1' in the export)
The second item (Yellow Truck in this case) corresponds with a 2 in the export.

So to translate from 1's and 2's (other buttons have more options) in the export we need to make a map
Please execute all cells in this notebook to convince yourself that the logic is being captured correctly, because it is this logic that is encoded in the 'requested_python' column of the crosswalk for >500 elements.
 


In [207]:
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile


In [208]:
#anyone know how to read this directly from the nihtoolbox desk?  
#readfromNIH='https://nihtoolbox.desk.com/customer/portal/kb_article_attachments/144125/original.xlsx?1562788410'
#in the meantime, read from a downloaded copy (replace the paths below with your own locations):
fpath_downloadedNIH='/home/petra/UbWinSharedSpace1/ccf-nda-behavioral/PycharmToolbox/Ipad2NDA_withCrosswalk/NIHToolbox2NDA/'
fname='NIH_Toolbox_IPAD_DataDictionary041119_Edited_070919_interpretted.xlsx'
fstring=fpath_downloadedNIH+fname

In [209]:
#read crosswalkfile (dont assume that all of the NIH Toolbox DD vars are complete - that code down stream of this in 
#iterative process)
pathin="/home/petra/UbWinSharedSpace1/ccf-nda-behavioral/PycharmToolbox/Ipad2NDA_withCrosswalk/NIHToolbox2NDA/"
crosswalkfile="Crosswalk_NIH_Toolbox_2_NDA.csv"
crosswalk=pd.read_csv(crosswalkfile,header=0,low_memory=False, encoding = "ISO-8859-1")
crosswalk.head()

#import csv
#with open(crosswalkfile, 'rb') as f:
#    reader = csv.reader(f)
#    linenumber = 1
#    try:
#        for row in reader:
#            linenumber += 1
#    except Exception as e:
#        print("Error line "+str(linenumber))
        


,Measurement System,Domain,Inst,Item ID,Stem,Context,DataType,Responses,Translation (output of IPAD),hcp_variable,...,specialty_code,hcp_variable_upload,nda_structure,nda_element,description,valueRange,notes,template,inst_short,Source
0,NIH Toolbox,Cognition,NIH Toolbox Picture Vocabulary Test Age 3+ v2.0,LAVOC188,LAVOC188,NaN,integer,0=LAVOC188-1\n0=LAVOC188-2\n1=LAVOC188-9\n0=LA...,NaN,lavoc188,...,button convert,lavoc188,tpvt01,lavoc188,Picture Vocabulary Item 188,0;1,0=Incorrect; 1=Correct,TPVT_v2.0.tpvt01_template.csv,TPVT_v2.0,HCPD HCPA
1,NIH Toolbox,Cognition,NIH Toolbox Picture Vocabulary Test Age 3+ v2.0,LAVOC189,LAVOC189,NaN,integer,1=LAVOC189-9\n0=LAVOC189-3\n0=LAVOC189-2\n0=LA...,NaN,lavoc189,...,button convert,lavoc189,tpvt01,lavoc189,Picture Vocabulary Item 189,0;1,0=Incorrect; 1=Correct,TPVT_v2.0.tpvt01_template.csv,TPVT_v2.0,HCPD
2,NIH Toolbox,Cognition,NIH Toolbox Picture Vocabulary Test Age 3+ v2.0,LAVOC330,LAVOC330,NaN,integer,0=LAVOC330-1\n1=LAVOC330-9\n0=LAVOC330-3\n0=LA...,NaN,lavoc330,...,button convert,lavoc330,tpvt01,lavoc330,Picture Vocabulary Item 330,0;1,0=Incorrect; 1=Correct,TPVT_v2.0.tpvt01_template.csv,TPVT_v2.0,HCPD
3,NIH Toolbox,Cognition,NIH Toolbox Picture Vocabulary Test Age 3+ v2.0,LAVOC120,LAVOC120,NaN,integer,0=LAVOC120-2\n0=LAVOC120-3\n1=LAVOC120-9\n0=LA...,NaN,lavoc120,...,button convert,lavoc120,tpvt01,lavoc120,Picture Vocabulary Item 120,0;1,0=Incorrect; 1=Correct,TPVT_v2.0.tpvt01_template.csv,TPVT_v2.0,HCPD
4,NIH Toolbox,Cognition,NIH Toolbox Picture Vocabulary Test Age 3+ v2.0,LAVOC259,LAVOC259,NaN,integer,0=LAVOC259-3\n0=LAVOC259-2\n0=LAVOC259-1\n1=LA...,NaN,lavoc259,...,button convert,lavoc259,tpvt01,lavoc259,Picture Vocabulary Item 259,0;1,0=Incorrect; 1=Correct,TPVT_v2.0.tpvt01_template.csv,TPVT_v2.0,HCPD


In [210]:
#print(fstring)
tlbxitems = pd.read_excel(fstring, sheet_name='NIH Toolbox')

print("Column headings:")
print(tlbxitems.columns)


Column headings:
Index(['Measurement System', 'Domain', 'Instrument Title', 'Item ID', 'Stem',
       'Context', 'DataType', 'Responses', 'Translation (output of IPAD)'],
      dtype='object')


In [211]:
tlbxitems['Item ID'].head()

0     VOCAB_INTRO
1    VOCAB_INSTR1
2    VOCAB_PRACT1
3    VOCAB_PRACT2
4    VOCAB_INSTR2
Name: Item ID, dtype: object

In [212]:
tlbxitems['Item ID'].shape

(4778,)

In [213]:
#to identify and create specialty button code to populate the 'Translation (output of IPAD)' column
#send this code back to the NIH Toolbox people so that they can update their data dictionary 
#and other people (who might not have yet identified this issue) don't have this problem

In [214]:
tlbxitems['hcp_variable']=tlbxitems['Item ID'].str.lower().str.replace('-','_').str.replace('(','_').str.replace(')','_')

In [219]:
#of the following instruments, specialty code to resolve button press issues needs to created for at
#least the DCCS instruments and the Picture Vocab Tests...lets start there
insts_w_buttons_issues =crosswalk.loc[crosswalk.specialty_code.str.contains('button')==True].Inst.unique().astype(list)
insts_w_buttons_issues

array(['NIH Toolbox Picture Vocabulary Test Age 3+ v2.0',
       'NIH Toolbox Pattern Comparison Processing Speed Test Ages 3 - 6 v2.1',
       'NIH Toolbox Pattern Comparison Processing Speed Test Age 7+ v2.1',
       'NIH Toolbox Flanker Inhibitory Control and Attention Test Ages 3-7 v2.1',
       'NIH Toolbox Flanker Inhibitory Control and Attention Test Ages 8-11 v2.1',
       'NIH Toolbox Flanker Inhibitory Control and Attention Test Age 12+ v2.1',
       'NIH Toolbox Dimensional Change Card Sort Test Ages 3-7 v2.1',
       'NIH Toolbox Dimensional Change Card Sort Test Ages 8-11 v2.1',
       'NIH Toolbox Dimensional Change Card Sort Test Age 12+ v2.1',
       'NIH Toolbox Picture Vocabulary Test Age 3+ v2.1',
       'NIH Toolbox Oral Reading Recognition Test Age 3+ v2.1'],
      dtype=object)

In [220]:
##subset for testing
#insts_w_buttons_issues =['NIH Toolbox Picture Vocabulary Test Age 3+ Practice v2.0',
#                         'NIH Toolbox Dimensional Change Card Sort Test Age 12+ v2.1',
#                         'NIH Toolbox Picture Vocabulary Test Age 3+ v2.0',
#                         'NIH Toolbox Pattern Comparison Processing Speed Test Age 7+ v2.1'
#                         NIH Toolbox Pattern Comparison Processing Speed Test Ages 3 - 6 v2.1
#                         ]

In [221]:
#examples (look at the Response column)
tlbxitems.loc[tlbxitems['Instrument Title']=='NIH Toolbox Dimensional Change Card Sort Test Age 12+ v2.1']
tlbxitems.loc[tlbxitems['Instrument Title']=='NIH Toolbox Picture Vocabulary Test Age 3+ v2.0']


,Measurement System,Domain,Instrument Title,Item ID,Stem,Context,DataType,Responses,Translation (output of IPAD),hcp_variable
5,NIH Toolbox,Cognition,NIH Toolbox Picture Vocabulary Test Age 3+ v2.0,LAVOC003,LAVOC003,NaN,integer,0=LAVOC003-3\n0=LAVOC003-2\n0=LAVOC003-1\n1=LA...,NaN,lavoc003
6,NIH Toolbox,Cognition,NIH Toolbox Picture Vocabulary Test Age 3+ v2.0,LAVOC051,LAVOC051,NaN,integer,1=LAVOC051-9\n0=LAVOC051-1\n0=LAVOC051-3\n0=LA...,NaN,lavoc051
7,NIH Toolbox,Cognition,NIH Toolbox Picture Vocabulary Test Age 3+ v2.0,LAVOC011,LAVOC011,NaN,integer,0=LAVOC011-3\n1=LAVOC011-9\n0=LAVOC011-1\n0=LA...,NaN,lavoc011
8,NIH Toolbox,Cognition,NIH Toolbox Picture Vocabulary Test Age 3+ v2.0,LAVOC014,LAVOC014,NaN,integer,0=LAVOC014-3\n0=LAVOC014-2\n0=LAVOC014-1\n1=LA...,NaN,lavoc014
9,NIH Toolbox,Cognition,NIH Toolbox Picture Vocabulary Test Age 3+ v2.0,LAVOC013,LAVOC013,NaN,integer,0=LAVOC013-1\n1=LAVOC013-9\n0=LAVOC013-3\n0=LA...,NaN,lavoc013
10,NIH Toolbox,Cognition,NIH Toolbox Picture Vocabulary Test Age 3+ v2.0,LAVOC028,LAVOC028,NaN,integer,0=LAVOC028-2\n0=LAVOC028-3\n0=LAVOC028-1\n1=LA...,NaN,lavoc028
11,NIH Toolbox,Cognition,NIH Toolbox Picture Vocabulary Test Age 3+ v2.0,LAVOC062,LAVOC062,NaN,integer,0=LAVOC062-2\n0=LAVOC062-1\n0=LAVOC062-3\n1=LA...,NaN,lavoc062
12,NIH Toolbox,Cognition,NIH Toolbox Picture Vocabulary Test Age 3+ v2.0,LAVOC004,LAVOC004,NaN,integer,1=LAVOC004-9\n0=LAVOC004-3\n0=LAVOC004-2\n0=LA...,NaN,lavoc004
13,NIH Toolbox,Cognition,NIH Toolbox Picture Vocabulary Test Age 3+ v2.0,LAVOC002,LAVOC002,NaN,integer,0=LAVOC002-2\n1=LAVOC002-9\n0=LAVOC002-3\n0=LA...,NaN,lavoc002
14,NIH Toolbox,Cognition,NIH Toolbox Picture Vocabulary Test Age 3+ v2.0,LAVOC008,LAVOC008,NaN,integer,1=LAVOC008-9\n0=LAVOC008-1\n0=LAVOC008-2\n0=LA...,NaN,lavoc008


In [222]:
#How to resolve?  Define a function that will translate this into code to populate the crosswalk
#I.E parsing the response into 'correct' and 'button' logic, and then rebuild that into a python snppit
#for any entry in the 'Response' column for a given row in the NIH Toolbox's data dictionary. 

#example:  
#A=buttoncorrect(instrument='NIH Toolbox Dimensional Change Card Sort Test Age 12+ v2.1',variable='dccs_shape_prac2')
#A is a tuple: ttestkeep,pythonsnp,translation
#A[0] is ttestkeep- is the mini data frame that illustrates the logic of the translation
#A[1] is pythonsnp- is the python code to translate NIH Toolbox to NDA
#A[2] is translation corresponds with the verbiage to add to the NIH Toolbox data dictionary for interpretation


def buttoncorrect(instrument='NIH Toolbox Dimensional Change Card Sort Test Age 12+ v2.1',variable='dccs_shape_instr1'):
    ddsub=tlbxitems.loc[tlbxitems['Instrument Title']==instrument].copy()
    test=ddsub.loc[ddsub.hcp_variable==variable].Responses.str.split('\n',expand=True)
    test=test.reset_index(drop=True)
    ttest=test.transpose()
    ttest['dd']=ttest[0]
    ttest['button']=ttest.index + 1
    ttestkeep=ttest.loc[ttest.dd.str.contains('=')==True,['dd','button']]
    ttestkeep['NDA_answer_as_string']=''
    ttestkeep.loc[ttest.dd.str.contains('0='),'NDA_answer_as_string']='incorrect'
    ttestkeep.loc[ttest.dd.str.contains('1='),'NDA_answer_as_string']='correct'
    ttestkeep.loc[ttest.dd.str.contains('0='),'NDA_answer_as_number']='0'
    ttestkeep.loc[ttest.dd.str.contains('1='),'NDA_answer_as_number']='1'
    #now we turn all this logic into a single string of code for the given response option 
    #that can be used to populate the 'requested_python' field of the crosswalk
    ttestkeep['codeblock']='studydata.loc[studydata.{}=='.format(variable)+ttestkeep.button.astype(str)+",'{}']=".format(variable) +ttestkeep.NDA_answer_as_number
    ttestkeep['translate']='IPAD output value of '+ttestkeep.button.astype(str)+'='+ttestkeep.NDA_answer_as_string
    pythonsnp=';'.join(ttestkeep.codeblock.tolist())
    translation=';'.join(ttestkeep.translate.tolist())
    return ttestkeep,pythonsnp,translation # you'll only actually need the pythonsnp, but prove to yourself that its working by examining the output for a few vars
    


In [223]:
#see that the logic is getting mapped properly
ttestkeep,pythonsnp,translation=buttoncorrect(instrument='NIH Toolbox Dimensional Change Card Sort Test Age 12+ v2.1',variable='dccs_shape_prac2')
print(translation)
ttestkeep

IPAD output value of 1=incorrect;IPAD output value of 2=correct


,dd,button,NDA_answer_as_string,NDA_answer_as_number,codeblock,translate
0,0=WhiteRabbit,1,incorrect,0,"studydata.loc[studydata.dccs_shape_prac2==1,'d...",IPAD output value of 1=incorrect
1,1=BrownBoat,2,correct,1,"studydata.loc[studydata.dccs_shape_prac2==2,'d...",IPAD output value of 2=correct


In [225]:
#see that the logic is getting translated to extractable code
ttestkeep,pythonsnp,translation=buttoncorrect(instrument='NIH Toolbox Picture Vocabulary Test Age 3+ v2.0',variable='lavoc091')
for i in ttestkeep.codeblock:
    print(i)
print("***")
print(pythonsnp)
print(translation)
ttestkeep

studydata.loc[studydata.lavoc091==1,'lavoc091']=0
studydata.loc[studydata.lavoc091==2,'lavoc091']=1
studydata.loc[studydata.lavoc091==3,'lavoc091']=0
studydata.loc[studydata.lavoc091==4,'lavoc091']=0
***
studydata.loc[studydata.lavoc091==1,'lavoc091']=0;studydata.loc[studydata.lavoc091==2,'lavoc091']=1;studydata.loc[studydata.lavoc091==3,'lavoc091']=0;studydata.loc[studydata.lavoc091==4,'lavoc091']=0
IPAD output value of 1=incorrect;IPAD output value of 2=correct;IPAD output value of 3=incorrect;IPAD output value of 4=incorrect


,dd,button,NDA_answer_as_string,NDA_answer_as_number,codeblock,translate
0,0=LAVOC091-2,1,incorrect,0,"studydata.loc[studydata.lavoc091==1,'lavoc091']=0",IPAD output value of 1=incorrect
1,1=LAVOC091-9,2,correct,1,"studydata.loc[studydata.lavoc091==2,'lavoc091']=1",IPAD output value of 2=correct
2,0=LAVOC091-3,3,incorrect,0,"studydata.loc[studydata.lavoc091==3,'lavoc091']=0",IPAD output value of 3=incorrect
3,0=LAVOC091-1,4,incorrect,0,"studydata.loc[studydata.lavoc091==4,'lavoc091']=0",IPAD output value of 4=incorrect


Now make a wrap around function...

Given name of an affected instrument, find the variables with button issues, and output dataframe
with Instrument name, 'Item ID', 'hcp_variable', pythonsnp, and translation 
then put the translation into the NIH DD (tlbxitems).  Put the pythonsnp into the crosswalk


In [229]:
#inst=insts_w_buttons_issues[2]
#print(inst)
for inst in insts_w_buttons_issues:
    ddsub=tlbxitems.loc[tlbxitems['Instrument Title']==inst].copy()
    if ddsub.empty:
        print(inst+ ' is not in the NIH Toolbox Data Dictionary so button issue cannot be resolved')
    else:
        ddsub_buttons=ddsub.loc[ddsub.Responses.str.contains('1=')==True,['Instrument Title','Item ID','Responses','hcp_variable']] 
        ddsub_buttons
        #remember that the output of the buttoncorrectfunction is a tuple
        #so the [2] below is where we grab the specific output of the function want and put it into the column of interest 
        #similarly, the [1] below is where we grab the python code
        ddsub_buttons['New Translation (output of IPAD)']=ddsub_buttons.apply(lambda x: buttoncorrect(instrument=inst,variable=x['hcp_variable'])[2],axis=1)
        ##ddsub_buttons['requested_python']=ddsub_buttons.apply(lambda x: buttoncorrect(instrument=inst,variable=x['hcp_variable'])[1],axis=1)
        tlbxitemstest=pd.merge(tlbxitems,
            ddsub_buttons,on=['Instrument Title','Item ID','Responses','hcp_variable'],how='left')
        tlbxitemstest.loc[tlbxitemstest['Translation (output of IPAD)'].isnull()==True,'Translation (output of IPAD)']=tlbxitemstest['New Translation (output of IPAD)']
        tlbxitems=tlbxitemstest.drop(columns='New Translation (output of IPAD)').copy()

NIH Toolbox Picture Vocabulary Test Age 3+ v2.1 is not in the NIH Toolbox Data Dictionary so button issue cannot be resolved
NIH Toolbox Oral Reading Recognition Test Age 3+ v2.1 is not in the NIH Toolbox Data Dictionary so button issue cannot be resolved


In [230]:
#Before:
#Column headings:
#Index(['Measurement System', 'Domain', 'Instrument Title', 'Item ID', 'Stem',
#       'Context', 'DataType', 'Responses', 'Translation (output of IPAD)'],
#      dtype='object')




#check that tlbxitems hasnt changed length
print(tlbxitems.shape)
print(tlbxitems.columns)

(4862, 10)
Index(['Measurement System', 'Domain', 'Instrument Title', 'Item ID', 'Stem',
       'Context', 'DataType', 'Responses', 'Translation (output of IPAD)',
       'hcp_variable'],
      dtype='object')


In [231]:
 tlbxitems.loc[tlbxitems['Translation (output of IPAD)'].isnull()==False]

,Measurement System,Domain,Instrument Title,Item ID,Stem,Context,DataType,Responses,Translation (output of IPAD),hcp_variable
5,NIH Toolbox,Cognition,NIH Toolbox Picture Vocabulary Test Age 3+ v2.0,LAVOC003,LAVOC003,NaN,integer,0=LAVOC003-3\n0=LAVOC003-2\n0=LAVOC003-1\n1=LA...,IPAD output value of 1=incorrect;IPAD output v...,lavoc003
6,NIH Toolbox,Cognition,NIH Toolbox Picture Vocabulary Test Age 3+ v2.0,LAVOC051,LAVOC051,NaN,integer,1=LAVOC051-9\n0=LAVOC051-1\n0=LAVOC051-3\n0=LA...,IPAD output value of 1=correct;IPAD output val...,lavoc051
7,NIH Toolbox,Cognition,NIH Toolbox Picture Vocabulary Test Age 3+ v2.0,LAVOC011,LAVOC011,NaN,integer,0=LAVOC011-3\n1=LAVOC011-9\n0=LAVOC011-1\n0=LA...,IPAD output value of 1=incorrect;IPAD output v...,lavoc011
8,NIH Toolbox,Cognition,NIH Toolbox Picture Vocabulary Test Age 3+ v2.0,LAVOC014,LAVOC014,NaN,integer,0=LAVOC014-3\n0=LAVOC014-2\n0=LAVOC014-1\n1=LA...,IPAD output value of 1=incorrect;IPAD output v...,lavoc014
9,NIH Toolbox,Cognition,NIH Toolbox Picture Vocabulary Test Age 3+ v2.0,LAVOC013,LAVOC013,NaN,integer,0=LAVOC013-1\n1=LAVOC013-9\n0=LAVOC013-3\n0=LA...,IPAD output value of 1=incorrect;IPAD output v...,lavoc013
10,NIH Toolbox,Cognition,NIH Toolbox Picture Vocabulary Test Age 3+ v2.0,LAVOC028,LAVOC028,NaN,integer,0=LAVOC028-2\n0=LAVOC028-3\n0=LAVOC028-1\n1=LA...,IPAD output value of 1=incorrect;IPAD output v...,lavoc028
11,NIH Toolbox,Cognition,NIH Toolbox Picture Vocabulary Test Age 3+ v2.0,LAVOC062,LAVOC062,NaN,integer,0=LAVOC062-2\n0=LAVOC062-1\n0=LAVOC062-3\n1=LA...,IPAD output value of 1=incorrect;IPAD output v...,lavoc062
12,NIH Toolbox,Cognition,NIH Toolbox Picture Vocabulary Test Age 3+ v2.0,LAVOC004,LAVOC004,NaN,integer,1=LAVOC004-9\n0=LAVOC004-3\n0=LAVOC004-2\n0=LA...,IPAD output value of 1=correct;IPAD output val...,lavoc004
13,NIH Toolbox,Cognition,NIH Toolbox Picture Vocabulary Test Age 3+ v2.0,LAVOC002,LAVOC002,NaN,integer,0=LAVOC002-2\n1=LAVOC002-9\n0=LAVOC002-3\n0=LA...,IPAD output value of 1=incorrect;IPAD output v...,lavoc002
14,NIH Toolbox,Cognition,NIH Toolbox Picture Vocabulary Test Age 3+ v2.0,LAVOC008,LAVOC008,NaN,integer,1=LAVOC008-9\n0=LAVOC008-1\n0=LAVOC008-2\n0=LA...,IPAD output value of 1=correct;IPAD output val...,lavoc008


In [232]:
#update the interpretted data dictionary by adding interpretations from this csv file to the github xlsx
fpath_downloadedNIH='/home/petra/UbWinSharedSpace1/ccf-nda-behavioral/PycharmToolbox/Ipad2NDA_withCrosswalk/NIHToolbox2NDA/'
newname='tempfile_interpretted_buttons.csv'
fstring=fpath_downloadedNIH+newname
tlbxitems.to_csv(fstring,index=False)

In [183]:
#now update the crosswalk with python snppits
#similar loop as before only this type we're updating crosswalk
#inst=insts_w_buttons_issues[2]
allbuttons=pd.DataFrame()
for inst in insts_w_buttons_issues :
    #subset to instrument
    ddsub=tlbxitems.loc[tlbxitems['Instrument Title']==inst].copy()
    #find the items that have responsese indicating button logic
    ddsub_buttons=ddsub.loc[ddsub.Responses.str.contains('1=')==True,['Instrument Title','Item ID','Responses','hcp_variable']] 
    #apply the button logic function
    ddsub_buttons['requested_python']=ddsub_buttons.apply(lambda x: buttoncorrect(instrument=inst,variable=x['hcp_variable'])[1],axis=1)
    #subset the output you want for merging with the crosswalk
    ddsub_buttons=ddsub_buttons[['Instrument Title','hcp_variable','requested_python']].rename(columns={'Instrument Title':'Inst'}).copy()
    allbuttons=pd.concat([allbuttons,ddsub_buttons],axis=0)

allbuttons

,Inst,hcp_variable,requested_python
2,NIH Toolbox Picture Vocabulary Test Age 3+ Pra...,vocab_pract1,"studydata.loc[studydata.vocab_pract1==1,'vocab..."
3,NIH Toolbox Picture Vocabulary Test Age 3+ Pra...,vocab_pract2,"studydata.loc[studydata.vocab_pract2==1,'vocab..."
2605,NIH Toolbox Dimensional Change Card Sort Test ...,dccs_shape_instr1,"studydata.loc[studydata.dccs_shape_instr1==1,'..."
2606,NIH Toolbox Dimensional Change Card Sort Test ...,dccs_shape_instr2,"studydata.loc[studydata.dccs_shape_instr2==1,'..."
2608,NIH Toolbox Dimensional Change Card Sort Test ...,dccs_shape_prac1,"studydata.loc[studydata.dccs_shape_prac1==1,'d..."
2609,NIH Toolbox Dimensional Change Card Sort Test ...,dccs_shape_prac2,"studydata.loc[studydata.dccs_shape_prac2==1,'d..."
2610,NIH Toolbox Dimensional Change Card Sort Test ...,dccs_shape_prac3,"studydata.loc[studydata.dccs_shape_prac3==1,'d..."
2611,NIH Toolbox Dimensional Change Card Sort Test ...,dccs_shape_prac4,"studydata.loc[studydata.dccs_shape_prac4==1,'d..."
2612,NIH Toolbox Dimensional Change Card Sort Test ...,dccs_shape_instr5,"studydata.loc[studydata.dccs_shape_instr5==1,'..."
2613,NIH Toolbox Dimensional Change Card Sort Test ...,dccs_shape_instr6,"studydata.loc[studydata.dccs_shape_instr6==1,'..."


In [189]:
#now merge this data into the crosswalk and write the new csv
#note you'll have to do some massaging to the crosswalk after it has been updated because there will be two requested_python
#vars now...preventing overwrite...need to open the file and update the cells by hand until see that no bugs here that would be 
#introduced by auto update
newcrosswalk=pd.merge(crosswalk,allbuttons,how='left',on=['Inst','hcp_variable'])

In [190]:
newcrosswalk.columns

Index(['Inst', 'template', 'inst_short', 'Source', 'nda_structure',
       'nda_element', 'hcp_variable', 'action_requested',
       'hcp_variable_upload', 'specialty_code', 'requested_python_x',
       'description', 'valueRange', 'notes', 'requested_python_y'],
      dtype='object')

In [191]:
newcrosswalk.to_csv(pathin+crosswalkfile,index=False)#crosswalkfile,index=False)